# Capstone 3 - Automating Deferral Discrepancy Report

# Preprocessing

In [1]:
#load modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce

In [2]:
# load data

df = pd.read_csv('../data/interim/DVdata.csv', index_col=0)
df.head()

,send,discrepancy_type,plan_cd,part_cd,pr_run_date,pr_post_date,pr_comp,pr_def_amt,ytd_def_amt,pretax_roth_cd,pr_def_pct,internal_pct_calc,diff_pr_internal_pct,rate_eff_date,rate_req_date,rate_req_origin_cd,rate_req_if_pct,rate_req_if_amt,plan_change_schedule_cd,annual_irs_limit
0,no,RateIssue_PR%_Low,1,1,1/8/2021,1/8/2021,2080.80,62.42,166.46,PreTax,3.00,6.00,-3.00,1/8/2021,1/8/2021,Participant Web,6.0,0.0,U,19500
1,no,RateIssue_PR%_Low,2,2,1/8/2021,1/8/2021,2273.08,272.77,272.77,PreTax,12.00,13.00,-1.00,1/7/2021,1/7/2021,CSR,13.0,0.0,U,19500
2,no,RateIssue_PR%_Low,3,3,1/7/2021,1/7/2021,10416.87,644.00,644.00,Roth,6.18,7.86,-1.68,1/7/2021,1/7/2021,Participant Web,0.0,819.0,U,19500
3,no,RateIssue_PR%_High,1,4,1/8/2021,1/8/2021,4615.38,692.31,692.31,Roth,15.00,6.00,9.00,1/7/2021,1/7/2021,Participant Web,6.0,0.0,U,19500
4,no,RateIssue_PR%_High,1,5,1/8/2021,1/8/2021,1846.16,129.23,129.23,PreTax,7.00,6.00,1.00,1/7/2021,1/7/2021,Participant Web,6.0,0.0,U,19500


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29473 entries, 0 to 32175
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   send                     29473 non-null  object 
 1   discrepancy_type         29473 non-null  object 
 2   plan_cd                  29473 non-null  int64  
 3   part_cd                  29473 non-null  int64  
 4   pr_run_date              29473 non-null  object 
 5   pr_post_date             29473 non-null  object 
 6   pr_comp                  29473 non-null  float64
 7   pr_def_amt               29473 non-null  float64
 8   ytd_def_amt              29473 non-null  float64
 9   pretax_roth_cd           29473 non-null  object 
 10  pr_def_pct               29473 non-null  float64
 11  internal_pct_calc        29473 non-null  float64
 12  diff_pr_internal_pct     29473 non-null  float64
 13  rate_eff_date            29473 non-null  object 
 14  rate_req_date         

In [4]:
# prepping df for model

df = df.drop(columns = [
    'pr_run_date', 
    'pr_post_date', 
    'rate_eff_date', 
    'rate_req_date', 
    'rate_req_origin_cd', 
    'plan_change_schedule_cd'
])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29473 entries, 0 to 32175
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   send                  29473 non-null  object 
 1   discrepancy_type      29473 non-null  object 
 2   plan_cd               29473 non-null  int64  
 3   part_cd               29473 non-null  int64  
 4   pr_comp               29473 non-null  float64
 5   pr_def_amt            29473 non-null  float64
 6   ytd_def_amt           29473 non-null  float64
 7   pretax_roth_cd        29473 non-null  object 
 8   pr_def_pct            29473 non-null  float64
 9   internal_pct_calc     29473 non-null  float64
 10  diff_pr_internal_pct  29473 non-null  float64
 11  rate_req_if_pct       29473 non-null  float64
 12  rate_req_if_amt       29473 non-null  float64
 13  annual_irs_limit      29473 non-null  int64  
dtypes: float64(8), int64(3), object(3)
memory usage: 3.4+ MB


In [5]:
# encoding 'send'

df['send'].value_counts()

yes    22309
no      7164
Name: send, dtype: int64

In [6]:
df['send'] = df['send'].map({'yes': 1, 'no': 0})
df.head()

,send,discrepancy_type,plan_cd,part_cd,pr_comp,pr_def_amt,ytd_def_amt,pretax_roth_cd,pr_def_pct,internal_pct_calc,diff_pr_internal_pct,rate_req_if_pct,rate_req_if_amt,annual_irs_limit
0,0,RateIssue_PR%_Low,1,1,2080.80,62.42,166.46,PreTax,3.00,6.00,-3.00,6.0,0.0,19500
1,0,RateIssue_PR%_Low,2,2,2273.08,272.77,272.77,PreTax,12.00,13.00,-1.00,13.0,0.0,19500
2,0,RateIssue_PR%_Low,3,3,10416.87,644.00,644.00,Roth,6.18,7.86,-1.68,0.0,819.0,19500
3,0,RateIssue_PR%_High,1,4,4615.38,692.31,692.31,Roth,15.00,6.00,9.00,6.0,0.0,19500
4,0,RateIssue_PR%_High,1,5,1846.16,129.23,129.23,PreTax,7.00,6.00,1.00,6.0,0.0,19500


In [7]:
# one-hot encoding 'discrepancy_type'

X_discrepancy = df['discrepancy_type']
ohe_discrepancy = ce.OneHotEncoder(cols=['discrepancy_type'])
X_discrepancy_ohe = ohe_discrepancy.fit_transform(X_discrepancy)

C:\Users\willi\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [8]:
df['discrepancy_type'].value_counts()

RateIssue_PR%_Low             11530
RateIssue_PR%_High             6439
AutoEnroll/Escalate_Missed     6216
CompIssue_PR%_Low              2764
CompIssue_PR%_High             1395
No_Salary_Reported             1113
Over_402g                        10
Catchup_Eligible                  6
Name: discrepancy_type, dtype: int64

In [9]:
X_discrepancy_ohe = X_discrepancy_ohe.rename(columns={
    'discrepancy_type_1':'rate_issue_pr%_low',
    'discrepancy_type_2':'rate_issue_pr%_high',
    'discrepancy_type_3':'autoenroll_autoescalate_missed',
    'discrepancy_type_4':'no_salary_reported',
    'discrepancy_type_5':'comp_issue_pr%_low',
    'discrepancy_type_6':'comp_issue_pr%_high',
    'discrepancy_type_7':'over_402g',
    'discrepancy_type_8':'catchup_eligible'})

df = pd.concat([df, X_discrepancy_ohe], axis=1)
df.head()

,send,discrepancy_type,plan_cd,part_cd,pr_comp,pr_def_amt,ytd_def_amt,pretax_roth_cd,pr_def_pct,internal_pct_calc,...,rate_req_if_amt,annual_irs_limit,rate_issue_pr%_low,rate_issue_pr%_high,autoenroll_autoescalate_missed,no_salary_reported,comp_issue_pr%_low,comp_issue_pr%_high,over_402g,catchup_eligible
0,0,RateIssue_PR%_Low,1,1,2080.80,62.42,166.46,PreTax,3.00,6.00,...,0.0,19500,1,0,0,0,0,0,0,0
1,0,RateIssue_PR%_Low,2,2,2273.08,272.77,272.77,PreTax,12.00,13.00,...,0.0,19500,1,0,0,0,0,0,0,0
2,0,RateIssue_PR%_Low,3,3,10416.87,644.00,644.00,Roth,6.18,7.86,...,819.0,19500,1,0,0,0,0,0,0,0
3,0,RateIssue_PR%_High,1,4,4615.38,692.31,692.31,Roth,15.00,6.00,...,0.0,19500,0,1,0,0,0,0,0,0
4,0,RateIssue_PR%_High,1,5,1846.16,129.23,129.23,PreTax,7.00,6.00,...,0.0,19500,0,1,0,0,0,0,0,0


In [10]:
# encoding 'pretax_roth_cd'

X_def_type = df['pretax_roth_cd']
ohe_def_type = ce.OneHotEncoder(cols=['pretax_roth_cd'])
X_def_type_ohe = ohe_def_type.fit_transform(X_def_type)

C:\Users\willi\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [11]:
X_def_type_ohe.value_counts()

pretax_roth_cd_1  pretax_roth_cd_2
1                 0                   23842
0                 1                    5631
dtype: int64

In [12]:
df['pretax_roth_cd'].value_counts()

PreTax    23842
Roth       5631
Name: pretax_roth_cd, dtype: int64

In [13]:
X_def_type_ohe = X_def_type_ohe.rename(columns={
    'pretax_roth_cd_1':'rate_type_pretax',
    'pretax_roth_cd_2':'rate_type_roth'})

df = pd.concat([df, X_def_type_ohe], axis=1)
df.head()

,send,discrepancy_type,plan_cd,part_cd,pr_comp,pr_def_amt,ytd_def_amt,pretax_roth_cd,pr_def_pct,internal_pct_calc,...,rate_issue_pr%_low,rate_issue_pr%_high,autoenroll_autoescalate_missed,no_salary_reported,comp_issue_pr%_low,comp_issue_pr%_high,over_402g,catchup_eligible,rate_type_pretax,rate_type_roth
0,0,RateIssue_PR%_Low,1,1,2080.80,62.42,166.46,PreTax,3.00,6.00,...,1,0,0,0,0,0,0,0,1,0
1,0,RateIssue_PR%_Low,2,2,2273.08,272.77,272.77,PreTax,12.00,13.00,...,1,0,0,0,0,0,0,0,1,0
2,0,RateIssue_PR%_Low,3,3,10416.87,644.00,644.00,Roth,6.18,7.86,...,1,0,0,0,0,0,0,0,0,1
3,0,RateIssue_PR%_High,1,4,4615.38,692.31,692.31,Roth,15.00,6.00,...,0,1,0,0,0,0,0,0,0,1
4,0,RateIssue_PR%_High,1,5,1846.16,129.23,129.23,PreTax,7.00,6.00,...,0,1,0,0,0,0,0,0,1,0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29473 entries, 0 to 32175
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   send                            29473 non-null  int64  
 1   discrepancy_type                29473 non-null  object 
 2   plan_cd                         29473 non-null  int64  
 3   part_cd                         29473 non-null  int64  
 4   pr_comp                         29473 non-null  float64
 5   pr_def_amt                      29473 non-null  float64
 6   ytd_def_amt                     29473 non-null  float64
 7   pretax_roth_cd                  29473 non-null  object 
 8   pr_def_pct                      29473 non-null  float64
 9   internal_pct_calc               29473 non-null  float64
 10  diff_pr_internal_pct            29473 non-null  float64
 11  rate_req_if_pct                 29473 non-null  float64
 12  rate_req_if_amt                 

In [15]:
# prepping final df for modelling

drop_columns = ['discrepancy_type', 'pretax_roth_cd', 'catchup_eligible', 'rate_type_roth']
df = df.drop(columns=drop_columns)
df.head()

,send,plan_cd,part_cd,pr_comp,pr_def_amt,ytd_def_amt,pr_def_pct,internal_pct_calc,diff_pr_internal_pct,rate_req_if_pct,rate_req_if_amt,annual_irs_limit,rate_issue_pr%_low,rate_issue_pr%_high,autoenroll_autoescalate_missed,no_salary_reported,comp_issue_pr%_low,comp_issue_pr%_high,over_402g,rate_type_pretax
0,0,1,1,2080.80,62.42,166.46,3.00,6.00,-3.00,6.0,0.0,19500,1,0,0,0,0,0,0,1
1,0,2,2,2273.08,272.77,272.77,12.00,13.00,-1.00,13.0,0.0,19500,1,0,0,0,0,0,0,1
2,0,3,3,10416.87,644.00,644.00,6.18,7.86,-1.68,0.0,819.0,19500,1,0,0,0,0,0,0,0
3,0,1,4,4615.38,692.31,692.31,15.00,6.00,9.00,6.0,0.0,19500,0,1,0,0,0,0,0,0
4,0,1,5,1846.16,129.23,129.23,7.00,6.00,1.00,6.0,0.0,19500,0,1,0,0,0,0,0,1


In [16]:
# saving data for modeling

df.to_csv('../data/processed/DVdata_processed.csv')